In [18]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark-3.0.3-bin-hadoop2.7'

In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("value_change_between").getOrCreate()

In [3]:
!pip install great_expectations==0.15.31

In [20]:
import datetime
 
import pandas as pd
from ruamel import yaml
 
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import (
    DataContextConfig,
    FilesystemStoreBackendDefaults,
)
 
root_directory = r"C:\Users\abhinav.anand\Desktop\custom_ge"
from great_expectations import *

In [21]:
data_context_config = DataContextConfig(
    store_backend_defaults=FilesystemStoreBackendDefaults(
        root_directory=root_directory
    ),
)
context = BaseDataContext(project_config=data_context_config)

In [22]:
path = "C:/Users/abhinav.anand/Desktop/custom_ge/tables/"
df = spark.read.format("csv").options(header = "true",inferschema = "true", sep=",",quote="\"",escape="\"").load(path+"sample_data.csv")
df.show()

+-----------+-------+--------------------+-------+--------------------+-----+----------+--------+--------+-------+-----+
|  spcl_char|no_char|               email|decimal|                 url| test|sample_col|test_int|store_id|revenue|check|
+-----------+-------+--------------------+-------+--------------------+-----+----------+--------+--------+-------+-----+
|r#st@et%^(=|   4785|abc.def@mail-arch...|   2.48|http://www.exampl...|  487|    215478|    4785|     715|   7800|    1|
|      ahbjs| 478stc|abc.def@mail#arch...|  2.147|http://www.exampl...|  487|   gfds784|     478|     715|   9500|    2|
|  r&cfr$5{>|gfds784|    abc.def@mail.org|   1.14|http://www.exampl...|     |       247|   78434|     715|   5100|    3|
|     sample| 215478|        abc.def@mail|  47.14|www.example.com/f...|  487|        70|  215478|     715|   5200|    4|
|     sample| 215478|        abc.def@mail|  12.11|www.example.com/f...|  487|        76|  215478|     615|   6300|    5|
|     sample| 215478|        abc

In [23]:
my_spark_datasource_config = {
    "name": "insert_your_datasource_name_here",
    "class_name": "Datasource",
    "execution_engine": {"class_name": "SparkDFExecutionEngine"},
    "data_connectors": {
        "insert_your_data_connector_name_here": {
            "module_name": "great_expectations.datasource.data_connector",
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": [
                "some_key_maybe_pipeline_stage",
                "some_other_key_maybe_run_id",
            ],
        }
    },
}
 
context.test_yaml_config(yaml.dump(my_spark_datasource_config))
 
context.add_datasource(**my_spark_datasource_config)
 
batch_request = RuntimeBatchRequest(
    datasource_name="insert_your_datasource_name_here",
    data_connector_name="insert_your_data_connector_name_here",
    data_asset_name="<YOUR_MEANGINGFUL_NAME>",  # This can be anything that identifies this data_asset for you
    batch_identifiers={
        "some_key_maybe_pipeline_stage": "prod",
        "some_other_key_maybe_run_id": f"my_run_name_{datetime.date.today().strftime('%Y%m%d')}",
    },
    runtime_parameters={"batch_data": df},  # Your dataframe goes here
)
 
expectation_suite_name = "insert_your_expectation_suite_name_here"
context.create_expectation_suite(
    expectation_suite_name=expectation_suite_name, overwrite_existing=True
)
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
)

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: SparkDFExecutionEngine
Data Connectors:
	insert_your_data_connector_name_here:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



In [9]:
# Databricks notebook source
import json
import re  # regular expressions

# !!! This giant block of imports should be something simpler, such as:
from great_expectations import *
from great_expectations.execution_engine import (
    PandasExecutionEngine,
    SparkDFExecutionEngine,
    SqlAlchemyExecutionEngine,
)
from great_expectations.expectations.expectation import (
    ColumnMapExpectation,
    Expectation,
    ExpectationConfiguration,
)
from great_expectations.expectations.metrics import (
    ColumnMapMetricProvider,
    column_condition_partial,
)
from great_expectations.expectations.registry import (
    _registered_expectations,
    _registered_metrics,
    _registered_renderers,
)
from great_expectations.expectations.util import render_evaluation_parameter_string
from great_expectations.render.renderer.renderer import renderer
from great_expectations.render.types import RenderedStringTemplateContent
from great_expectations.render.util import num_to_str, substitute_none_for_missing
from great_expectations.validator.validator import Validator

EMAIL_REGEX = r"[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,7}$"


class ColumnValuesContainValidEmail(ColumnMapMetricProvider):
    # This is the id string that will be used to reference your metric.
    condition_metric_name = "column_values.valid_email"
    condition_value_keys = ()

    # This method defines the business logic for evaluating your metric when using a PandasExecutionEngine
    @column_condition_partial(engine=PandasExecutionEngine)
    def _pandas(cls, column, **kwargs):
        def matches_email_regex(x):
            if re.match(EMAIL_REGEX, str(x)):
                return True
            return False

        return column.apply(lambda x: matches_email_regex(x) if x else False)

    # This method defines the business logic for evaluating your metric when using a SqlAlchemyExecutionEngine
    #     @column_condition_partial(engine=SqlAlchemyExecutionEngine)
    #     def _sqlalchemy(cls, column, _dialect, **kwargs):
    #         return column.in_([3])

    # This method defines the business logic for evaluating your metric when using a SparkDFExecutionEngine
    @column_condition_partial(engine=SparkDFExecutionEngine)
    def _spark(cls, column, **kwargs):
        return column.rlike(EMAIL_REGEX)


# This class defines the Expectation itself
# The main business logic for calculation lives here.
class ExpectColumnValuesToContainValidEmail(ColumnMapExpectation):
    # These examples will be shown in the public gallery, and also executed as unit tests for your Expectation
    examples = [
        {
            "data": {
                "fail_case_1": ["a123@something", "a123@something.", "a123."],
                "fail_case_2": ["aaaa.a123.co", "aaaa.a123.", "aaaa.a123.com"],
                "fail_case_3": ["aaaa@a123.e", "aaaa@a123.a", "aaaa@a123.d"],
                "fail_case_4": ["@a123.com", "@a123.io", "@a123.eu"],
                "pass_case_1": [
                    "a123@something.com",
                    "vinod.km@something.au",
                    "this@better.work",
                ],
                "pass_case_2": [
                    "example@website.dom",
                    "ex.ample@example.ex",
                    "great@expectations.email",
                ],
                "valid_emails": [
                    "Janedoe@company.org",
                    "someone123@stuff.net",
                    "mycompany@mycompany.com",
                ],
                "bad_emails": ["Hello, world!", "Sophia", "this should fail"],
            },
            "tests": [
                {
                    "title": "negative_test_for_no_domain_name",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "fail_case_1"},
                    "out": {
                        "success": False,
                        "unexpected_index_list": [0, 1, 2],
                        "unexpected_list": [
                            "a123@something",
                            "a123@something.",
                            "a123.",
                        ],
                    },
                },
                {
                    "title": "negative_test_for_no_at_symbol",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "fail_case_2"},
                    "out": {
                        "success": False,
                        "unexpected_index_list": [0, 1, 2],
                        "unexpected_list": [
                            "aaaa.a123.co",
                            "aaaa.a123.",
                            "aaaa.a123.com",
                        ],
                    },
                },
                {
                    "title": "negative_test_for_ending_with_one_character",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "fail_case_3"},
                    "out": {
                        "success": False,
                        "unexpected_index_list": [0, 1, 2],
                        "unexpected_list": [
                            "aaaa@a123.e",
                            "aaaa@a123.a",
                            "aaaa@a123.d",
                        ],
                    },
                },
                {
                    "title": "negative_test_for_emails_with_no_leading_string",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "fail_case_4"},
                    "out": {
                        "success": False,
                        "unexpected_index_list": [0, 1, 2],
                        "unexpected_list": [
                            "aaaa@a123.e",
                            "aaaa@a123.a",
                            "aaaa@a123.d",
                        ],
                    },
                },
                {
                    "title": "pass_test",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "pass_case_1"},
                    "out": {
                        "success": True,
                        "unexpected_index_list": [],
                        "unexpected_list": [],
                    },
                },
                {
                    "title": "pass_test",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "pass_case_2"},
                    "out": {
                        "success": True,
                        "unexpected_index_list": [],
                        "unexpected_list": [],
                    },
                },
                {
                    "title": "valid_emails",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "valid_emails"},
                    "out": {
                        "success": True,
                        "unexpected_index_list": [],
                        "unexpected_list": [],
                    },
                },
                {
                    "title": "invalid_emails",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "bad_emails"},
                    "out": {
                        "success": False,
                        "unexpected_index_list": [0, 1, 2],
                        "unexpected_list": [
                            "Hello, world!",
                            "Sophia",
                            "this should fail",
                        ],
                    },
                },
            ],
        }
    ]

    # This dictionary contains metadata for display in the public gallery
    library_metadata = {
        "maturity": "experimental",
        "tags": ["experimental", "column map expectation"],
        "contributors": [  # Github
            "@aworld1",
            "@enagola",
            "@spencerhardwick",
            "@vinodkri1",
            "@degulati",
            "@ljohnston931",
            "@rexboyce",
            "@lodeous",
            "@sophiarawlings",
            "@vtdangg",
        ],
    }

    # This is the id string of the Metric used by this Expectation.
    # For most Expectations, it will be the same as the `condition_metric_name` defined in your Metric class above.
    map_metric = "column_values.valid_email"

    # This is a list of parameter names that can affect whether the Expectation evaluates to True or False
    # Please see {some doc} for more information about domain and success keys, and other arguments to Expectations
    success_keys = ()

    # This dictionary contains default values for any parameters that should have default values
    default_kwarg_values = {}

    # This method defines a question Renderer
    # For more info on Renderers, see {some doc}
    # !!! This example renderer should render RenderedStringTemplateContent, not just a string


#     @classmethod
#     @renderer(renderer_type="renderer.question")
#     def _question_renderer(
#         cls, configuration, result=None, language=None, runtime_configuration=None
#     ):
#         column = configuration.kwargs.get("column")
#         mostly = configuration.kwargs.get("mostly")

#         return f'Do at least {mostly * 100}% of values in column "{column}" equal 3?'

# This method defines an answer Renderer
# !!! This example renderer should render RenderedStringTemplateContent, not just a string
#     @classmethod
#     @renderer(renderer_type="renderer.answer")
#     def _answer_renderer(
#         cls, configuration=None, result=None, language=None, runtime_configuration=None
#     ):
#         column = result.expectation_config.kwargs.get("column")
#         mostly = result.expectation_config.kwargs.get("mostly")
#         regex = result.expectation_config.kwargs.get("regex")
#         if result.success:
#             return f'At least {mostly * 100}% of values in column "{column}" equal 3.'
#         else:
#             return f'Less than {mostly * 100}% of values in column "{column}" equal 3.'

# This method defines a prescriptive Renderer
#     @classmethod
#     @renderer(renderer_type="renderer.prescriptive")
#     @render_evaluation_parameter_string
#     def _prescriptive_renderer(
#         cls,
#         configuration=None,
#         result=None,
#         language=None,
#         runtime_configuration=None,
#         **kwargs,
#     ):
# !!! This example renderer should be shorter
#         runtime_configuration = runtime_configuration or {}
#         include_column_name = runtime_configuration.get("include_column_name", True)
#         include_column_name = (
#             include_column_name if include_column_name is not None else True
#         )
#         styling = runtime_configuration.get("styling")
#         params = substitute_none_for_missing(
#             configuration.kwargs,
#             ["column", "regex", "mostly", "row_condition", "condition_parser"],
#         )

#         template_str = "values must be equal to 3"
#         if params["mostly"] is not None:
#             params["mostly_pct"] = num_to_str(
#                 params["mostly"] * 100, precision=15, no_scientific=True
#             )
#             # params["mostly_pct"] = "{:.14f}".format(params["mostly"]*100).rstrip("0").rstrip(".")
#             template_str += ", at least $mostly_pct % of the time."
#         else:
#             template_str += "."

#         if include_column_name:
#             template_str = "$column " + template_str

#         if params["row_condition"] is not None:
#             (
#                 conditional_template_str,
#                 conditional_params,
#             ) = parse_row_condition_string_pandas_engine(params["row_condition"])
#             template_str = conditional_template_str + ", then " + template_str
#             params.update(conditional_params)

#         return [
#             RenderedStringTemplateContent(
#                 **{
#                     "content_block_type": "string_template",
#                     "string_template": {
#                         "template": template_str,
#                         "params": params,
#                         "styling": styling,
#                     },
#                 }
#             )
#         ]

if __name__ == "__main__":
    ExpectColumnValuesToContainValidEmail().print_diagnostic_checklist()

# COMMAND ----------




Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "C:\Users\abhinav.anand\Anaconda3\envs\pysparkenv\lib\site-packages\great_expectations\self_check\util.py", line 2774, in evaluate_json_test_v3_api
    check_json_test_result(
  File "C:\Users\abhinav.anand\Anaconda3\envs\pysparkenv\lib\site-packages\great_expectations\self_check\util.py", line 2941, in check_json_test_result
    assert result["result"]["unexpected_list"] == value, (
AssertionError: expected ['aaaa@a123.a', 'aaaa@a123.d', 'aaaa@a123.e'] but got ['@a123.com', '@a123.eu', '@a123.io']



...

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

3 5 3 False
Completeness checklist for ExpectColumnValuesToContainValidEmail (EXPERIMENTAL):
 ✔ Has a valid library_metadata object
   Has a docstring, including a one-line short description
   Has at least one positive and negative example case, and all test cases pass
   Has core logic and passes tests on at least one Execution Engine
   Passes all linting checks
      inspect.getfile(impl) raised a TypeError (impl is a built-in class)
   Has basic input validation and type checking
      No validate_configuration method defined on subclass
 ✔ Has both statement Renderers: prescriptive and diagnostic
   Has core logic that passes tests for all applicable Execution Engines and SQL dialects
      Only 6 / 8 tests for pandas are passing
        - Failing: negative_test_for_emails_with_no_leading_string, valid_emails
      Only 7 / 8 tests for spark are passing
        - Failing: negative_test_for_emails_with_no_leading_string
   Has a full suite of tests, as determined by a code owner
 

In [25]:
validator.expect_column_values_to_contain_valid_email(column="email")
# result_format = {'result_format' : "SUMMARY"}

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

{
  "result": {
    "element_count": 16,
    "unexpected_count": 14,
    "unexpected_percent": 93.33333333333333,
    "partial_unexpected_list": [
      "abc.def@mail-archive.com",
      "abc.def@mail#archive.com",
      "abc.def@mail",
      "abc.def@mail",
      "abc.def@mail",
      "abc.def@mail",
      "abc.def@mail",
      "abc.def@mail",
      "abc.def@mail",
      "abc.def@mail",
      "abc.def@mail",
      "abc.def@mail",
      "abc.def@mail",
      "abc.def@mail"
    ],
    "missing_count": 1,
    "missing_percent": 6.25,
    "unexpected_percent_total": 87.5,
    "unexpected_percent_nonmissing": 93.33333333333333
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": false,
  "meta": {}
}

In [8]:
import psutil
psutil.cpu_count()

8

In [1]:
import posixpath as pp

In [3]:
class Constants:
    """This class defines all the constant variables used in the application
    """
    MANDATORY_FIELDS = ['database', 'table_names']
    DEFAULT_OPTIONAL_ARGS = {}
    GE_CONFIG_TBL_NAME = 'ge_config'
    BAD_RESULTS_PATH = 'bad_results'
    EXPECTATION_RESULTS_PREFIX = 'ge_expectation_results_json'
    GE_DC_EXPECTATION_PREFIX = 'expectations'
    GE_DC_VALIDATION_PREFIX = 'validations'
    GE_DC_DATA_DOCS_PREFIX = 'data_docs'


In [45]:
from datetime import datetime
today = datetime.date.today()
current_date_str = today.strftime('%Y%m%d')
current_timestamp_str = today.strftime('%H%M%S')
bucket_name = "ge"
db_root_prefix = "dq"
db_name = "star"
tbl_name = "actives"

AttributeError: 'method_descriptor' object has no attribute 'today'

In [35]:
bad_results_path = pp.join('s3://', bucket_name, db_root_prefix, db_name,
                           Constants.BAD_RESULTS_PATH, tbl_name, current_date_str,
                           tbl_name + '_' + current_timestamp_str + '.csv')

In [36]:
print(bad_results_path)

s3://ge/dq/star/bad_results/actives/20220817/actives_125353.csv


In [43]:
test = "C:/Users/abhinav.anand/Desktop/dq_check"+"/"+current_date_str+'_' + current_timestamp_str +".csv"
#"current_date_str"+".csv"

In [44]:
print(test)

C:/Users/abhinav.anand/Desktop/dq_check/20220817_125353.csv


In [49]:
data_df = spark.read.format("csv").options(inferSchema = "true",header = "true", sep= ",",quote="\"",
                                                                                  escape="\"").load(r"C:\Users\abhinav.anand\Downloads\sample_data_dq.csv")

In [50]:
data_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- url: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- created_ts: string (nullable = true)



In [1]:
a=7
b=6
print(a+b)

13
